# First we will convert an Explainable SageMaker Image Classification Model

In [1]:
blob_storage_provider = "S3"
blob_storage_container = "modzy-engineering-tests"
resources_key = "ds/model-converter/sagemaker/image-classification/resources.tar.gz"
weights_key = "ds/model-converter/sagemaker/image-classification/weights.tar.gz"

In [2]:
from modzy.converter.model_converter import ModelConverter
from modzy.client import ApiClient
import os

# To get started, store your Modzy API key as an environment variable `MODZY_API_KEY`.
# Then, create a Modzy API client to interact with the integration envrionment
modzy_api_key = os.getenv("MODZY_QA_API_KEY")
modzy_instance_base_url = "https://integration.modzy.engineering/api"
modzy_api_client = ApiClient(api_key=modzy_api_key, base_url=modzy_instance_base_url)

# Instantiate a Model Converter client with access to the Modzy integration environment
model_converter = ModelConverter(modzy_api_client)

In [3]:
# Now, provide the Model converter with information about your stored model assets and the credentials required
# to access them. The Model converter will do the rest of the work.

source_platform = "sagemaker"
model_type = "image-classification"

_, converter_output = model_converter.run(
        sp_access_key_id=os.getenv("SP_ACCESS_KEY_ID"),
        sp_secret_access_key=os.getenv("SP_SECRET_ACCESS_KEY"),
        blobstore_provider=blob_storage_provider,
        blobstore_container=blob_storage_container,
        weights_path=weights_key,
        resources_path=resources_key,
        platform=source_platform,
        model_type=model_type,
)

print(f"The model details page for your new model can be found here: {converter_output['modelURL']}")
new_model_id = converter_output["modelId"]
new_model_version = converter_output["modelVersion"]

The model details page for your new model can be found here: https://integration.modzy.engineering/models/106d50ca0e-modzy-image-classification/0.0.1


In [4]:
# Delegate a single processing to serve your new model
modzy_api_client.models.update_processing_engines(new_model_id, new_model_version, min_engines=1, max_engines=1)

In [ ]:
# Send an inference job to run against your new model with explainability!
input_source = {
    "0001": {
        f"image": {
            "bucket": blob_storage_container,
            "key": f"/ds/model-converter/{source_platform}/{model_type}/test_input"
        }
    }
}

print(f"Sending job to model {new_model_id} {new_model_version}")
job = modzy_api_client.jobs.submit_aws_s3(
    new_model_id, new_model_version, input_source,
    os.getenv("SP_ACCESS_KEY_ID"), os.getenv("SP_SECRET_ACCESS_KEY"),
    region="us-east-1", explain=True
)

modzy_api_client.jobs.block_until_complete(job, timeout=None)
print("Job Completed!")

# Next, we will convert an MLflow model

In [ ]:
from modzy.converter.mlflow import upload_mlflow_model

# Raw output MLFlow Model Gets uploaded to the user's S3 Bucket
# upload_mlflow_model()

# Now we repeat the process with an MLFlow model
source_platform = "mlflow"
model_type = "tabular"
resources_key = "ds/model-converter/mlflow/tabular/resources.tar.gz"
weights_key = "ds/model-converter/mlflow/tabular/model.tar.gz"


_, converter_output = model_converter.run(
        sp_access_key_id=os.getenv("SP_ACCESS_KEY_ID"),
        sp_secret_access_key=os.getenv("SP_SECRET_ACCESS_KEY"),
        blobstore_provider=blob_storage_provider,
        blobstore_container=blob_storage_container,
        weights_path=weights_key,
        resources_path=resources_key,
        platform=source_platform,
        model_type=model_type,
)

print(f"The model details page for your new model can be found here: {converter_output['modelURL']}")
new_model_id = converter_output["modelId"]
new_model_version = converter_output["modelVersion"]

In [ ]:
# Delegate a single processing to serve your new model
modzy_api_client.models.update_processing_engines("106d50ca0e", "0.0.1", min_engines=0, max_engines=1)